In [79]:
import json
import pandas as pd
import numpy as np
from pathlib import Path 
import re
import ast
import gc

In [80]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"

input_directory = Path(f'{main_path}Youtube/Input_test')  
output_directory = Path(f'{main_path}Youtube/Output')  

for file in output_directory.iterdir():
   if file.is_file():
      file.unlink() 

max_columns = 8


In [81]:
for i in range(max_columns):
        col_path = [f"col_path_{i}" for i in range(1, max_columns)]
        col_path_values = [f"col_path_{i}_values" for i in range(1, max_columns)]
        col_path_list = [f"col_path_{i}_lIST" for i in range(1, max_columns)]

In [82]:
print(col_path_values)

['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values', 'col_path_6_values', 'col_path_7_values']


### loading_data()

In [83]:
def loading_data(data):
    

    with open(data, 'r') as file:
        data = json.load(file)

    print(type(data))



    df = pd.DataFrame()
    df['col_path_0_values'] = data
    df['json_name'] = df.index
    df['json_name'] = df['json_name'].str.rsplit("/", n=1).str[-1]
    df['file_path'] = df.index
    
   
    df['col_path_0_LIST'] = ''



    col = df.pop('json_name') 
    df.insert(0, 'json_name', col)
   
    col = df.pop('file_path') 
    df.insert(0, 'file_path', col)
    
    df.reset_index(inplace = True)
    df = df.drop('index', axis = 1)

   
    
    for i in range(1,max_columns):   
        df[f'col_path_{i}'] = ''
        df[f'col_path_{i}_values'] = ''
        df[f'col_path_{i}_LIST'] = ''

    index_list = []
    for idx, row in df.iterrows():
        if row['col_path_0_values'] == 'No data':
            index_list.append(idx)
    
    df_no_data = df.loc[index_list]
    df = df.drop(index_list)
   

    
    return(df, df_no_data)



### flatten_json()

In [84]:
def flatten_json(df):
    
    new_rows = []
    current_rows = df.copy()  # snapshot to iterate over

    for ix, row in current_rows.iterrows():
        value= row['col_path_0_values']
            
        if isinstance(value, list):
                    # Check if list of dicts
            if value and isinstance(value[0], dict):
                
                
                seen = set()
                unique = []
                for d in value:
                    # Use json.dumps to get a hashable, sorted representation
                    s = json.dumps(d, sort_keys=True)
                    if s not in seen:
                        seen.add(s)
                        unique.append(d)
                value = unique
                

                for item in value:
                    new_row = row.copy()
                    new_row['col_path_0_values'] = item
                    new_row['col_path_0_LIST'] = 'LIST'
                    new_rows.append(new_row)

        else:
            df.at[ix, 'col_path_0_LIST'] = 'NO LIST'

    



    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    
    new_rows.clear()

           

    new_rows = []

    for i in range(max_columns):
        current_rows = df.copy()  # snapshot to iterate over

        for ix, row in current_rows.iterrows():
            value = row.get(f'col_path_{i}_values', None)
    

            if isinstance(value, dict):
                for k, v in value.items():
                    if isinstance(v, list):
                        # Check if list of dicts
                        if v and isinstance(v[0], dict):

                            seen = set()
                            unique = []
                            for d in v:
                                # Use json.dumps to get a hashable, sorted representation
                                s = json.dumps(d, sort_keys=True)
                                if s not in seen:
                                    seen.add(s)
                                    unique.append(d)
                            v = unique
                            

                            for item in v:
                                new_row = row.copy()
                                new_row[f'col_path_{i+1}_values'] = item
                                new_row[f'col_path_{i+1}'] = k
                                new_row[f'col_path_{i+1}_LIST'] = 'LIST'
                                new_rows.append(new_row)
                        else:
                            
                            v = v[0]
                            new_row = row.copy()
                            new_row[f'col_path_{i+1}_values'] = v
                            new_row[f'col_path_{i+1}'] = k
                            new_row[f'col_path_{i+1}_LIST'] = 'LIST'
                            new_rows.append(new_row)
                    else:
                
                        new_row = row.copy()
                        new_row[f'col_path_{i+1}_values'] = v
                        new_row[f'col_path_{i+1}'] = k
                        new_row[f'col_path_{i+1}_LIST'] = 'NO LIST'
                        new_rows.append(new_row)
                
                    if isinstance(v, str):
                        new_row = row.copy()
                        new_row[f'col_path_{i+1}_LIST'] = 'NO LIST'
                        new_rows.append(new_row)
                
                

        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
        new_rows.clear()
            
                        


    

    df = df.replace(r'^\s*$', np.nan, regex=True)
    df_red = df.drop(columns=[col for col in df.columns if col.endswith("_LIST")])
    
    df_red['last_valid_index'] = df_red.apply(pd.Series.last_valid_index, axis=1)
    df_red = df_red.drop(columns=[col for col in df.columns if col.endswith("values")])

    
    col_path = [f"col_path_{i}" for i in range(1, max_columns)]
    col_path = col_path + ['json_name'] + ['file_path'] 
    df = pd.merge(df, df_red, on = col_path, how='left')
    
    
    return(df)

### process_col_path()
The 'process_col_path()' function checks whether the value in a row of for one of the column paths is actually a datatype and stores this value in a column data_type and replaces the original value with NA. The data types are the lowest level values in the JSON files

In [85]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']



# Define the function 'process_col_path()'
def process_col_path(df, col_path_values, data_types):
    df['data_type'] = ''

    """
    row: Rows in the dataframe
    columns: List of column names of column path columns 
    data_types: List of the values that are data types
    """
    
    for ix, row in df.iterrows():
        for col in col_path_values:
            

            #If the value stored in the column is found in the list 'data_types', 
            if row[col] in data_types:
                # this value is placed in the column 'data_type'
                value = row[col]
                
                df.at[ix, 'data_type'] = value
            else:
                continue
               
    
                
    return df

### remove_intermediate_rows()

In [86]:
def remove_intermediate_rows(df):
    drop_index = []
    for ix, row in df.iterrows():
            last_value = row[f"{row['last_valid_index']}"]
           
        
            if isinstance(last_value, str):
                if last_value not in (data_types):

                        drop_index.append(ix)
            else:
                drop_index.append(ix)
                  
        
   
    df = df.drop(drop_index)
    df = df.drop(columns=[col for col in df.columns if col.endswith("values")])

    
    return df

### extract_path()

In [87]:
def extract_path(df, max_columns):
    
    df['path']= ''
    
    for ix, row in df.iterrows():
        path = []
    
        for col in [f"col_path_{i}" for i in range(1, max_columns)]:
            val = row[col]
        
            if pd.notna(val):

                path.append(str(val))

        
        df.at[ix, 'path'] = path
       
    
        

    return df




### list_path()

In [88]:
def list_path(df):
    
    df['list_path'] = ''
    df['subfield_path'] = ''
    df['column_name'] = ''
    df['var_type'] = ''
    
    new_rows = []
    delete_rows = []

    for ix, row in df.iterrows():
        
        path = row['path'] 
        found_list = False
        
        
        for i in reversed(range(len(path))):
           
            if row[f"col_path_{i}_LIST"] == 'LIST':
                
                new_row = row.copy()
                delete_rows.append(ix)
                new_row['var_type'] = 'list'
                new_row['list_path'] = path[:i]
                new_row['subfield_path'] = path[i:]
                new_row['column_name'] = path[-1]
                new_rows.append(new_row)

                found_list = True
                break

            elif row[f"col_path_{i}_LIST"] == "NO LIST":
                var_type = 'static'
                subfield_path = path
                column_name = path[-1]
                

               
            else:
                var_type = 'skip'
                subfield_path = path
                column_name = path[-1]


        if not found_list:  
            df.at[ix, 'subfield_path'] = subfield_path
            df.at[ix, 'column_name'] = column_name
            df.at[ix, 'var_type'] = var_type

    df = df.drop(index=delete_rows).reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    new_rows.clear()
    
        


      
        
    return df

### clean_and_store()
This function orders the columns in the DataFrame, resets the index, fills the NA and saves the DataFrame as CSV

In [89]:
def clean_and_store(df, df_no_data, file_name):

    """
    df: The dataframe that will be cleaned and stored
    file_name: The filename of data structure that is being processed
    """

  
   
    #df=  df[df['col_path_1'] != 'Direct Message']
    
    df = pd.concat([df, df_no_data], ignore_index=True)
    
  
    df = df[['json_name','column_name', 'path', 'list_path', 'subfield_path', 'var_type', 'data_type', 'file_path']]
    df = df.astype(str)
    df = df.drop_duplicates()
    # Save the DataFrame 
    df.to_csv(f"{main_path}Youtube/Output/Output_" + file_name + '.csv', index=False)

    return df

### structure_donations()
The structure_donations() function executes all functions above and results in a saved DataFrame for each data structure.

In [90]:
#pd.set_option('display.max_colwidth', 100)

def structure_donations(data, col_path, max_columns):
     # Store the path to the data structure
    data = Path(data)  
    
    # Save teh file name of the data structure
    file_name = Path(data).stem 

    df, df_no_data = loading_data(data)
    print('FINISH: loading_data()')
    df = flatten_json(df)
    print('FINISH: flatten_json()')
    df = process_col_path(df, col_path_values, data_types)
    print('FINISH: process_col_path()')
    df = remove_intermediate_rows(df)
    print('FINISH: remove_intermediate_rows()')
    df = extract_path(df, max_columns)
    print('FINISH: extract_path()')
    display(df)
    df = list_path(df)
    print('FINISH: list_path()')
    df = clean_and_store(df, df_no_data, file_name)
    print('FINISH: clean_and_store()')

    del df,df_no_data, data

    
    
    

    

## Execute 'structure_donations()': Transform data structures from JSON format to tabular format

In [91]:
# Execute the 'structure_donations()' function for each file (data structure) in the input directory
for file in input_directory.iterdir():  
    if file.is_file():  
        print("FILE NAME:", file.name)
        result = structure_donations(file, col_path, max_columns)
        del result
        gc.collect()
        

FILE NAME: YT_structure_youtube_takeout_es.json
<class 'dict'>
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()


/tmp/ipykernel_100495/546882260.py:107: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


,file_path,json_name,col_path_0_LIST,col_path_1,col_path_1_LIST,col_path_2,col_path_2_LIST,col_path_3,col_path_3_LIST,col_path_4,col_path_4_LIST,col_path_5,col_path_5_LIST,col_path_6,col_path_6_LIST,col_path_7,col_path_7_LIST,last_valid_index,data_type,path
32,historial/historial-de-búsqueda.json,historial-de-búsqueda.json,LIST,header,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[header]
41,historial/historial-de-búsqueda.json,historial-de-búsqueda.json,LIST,title,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[title]
50,historial/historial-de-búsqueda.json,historial-de-búsqueda.json,LIST,titleUrl,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[titleUrl]
59,historial/historial-de-búsqueda.json,historial-de-búsqueda.json,LIST,time,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[time]
68,historial/historial-de-búsqueda.json,historial-de-búsqueda.json,LIST,products,LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[products]
77,historial/historial-de-búsqueda.json,historial-de-búsqueda.json,LIST,activityControls,LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[activityControls]
86,historial/historial-de-reproducciones.json,historial-de-reproducciones.json,LIST,header,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[header]
95,historial/historial-de-reproducciones.json,historial-de-reproducciones.json,LIST,title,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[title]
104,historial/historial-de-reproducciones.json,historial-de-reproducciones.json,LIST,titleUrl,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[titleUrl]
113,historial/historial-de-reproducciones.json,historial-de-reproducciones.json,LIST,time,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[time]


FINISH: list_path()
FINISH: clean_and_store()
FILE NAME: YT_structure_youtube_takeout_dutch.json
<class 'dict'>
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()


/tmp/ipykernel_100495/546882260.py:107: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


...

,file_path,json_name,col_path_0_LIST,col_path_1,col_path_1_LIST,col_path_2,col_path_2_LIST,col_path_3,col_path_3_LIST,col_path_4,col_path_4_LIST,col_path_5,col_path_5_LIST,col_path_6,col_path_6_LIST,col_path_7,col_path_7_LIST,last_valid_index,data_type,path
32,istoric/istoricul căutărilor.json,istoricul căutărilor.json,LIST,header,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[header]
41,istoric/istoricul căutărilor.json,istoricul căutărilor.json,LIST,title,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[title]
50,istoric/istoricul căutărilor.json,istoricul căutărilor.json,LIST,titleUrl,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[titleUrl]
59,istoric/istoricul căutărilor.json,istoricul căutărilor.json,LIST,time,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[time]
68,istoric/istoricul căutărilor.json,istoricul căutărilor.json,LIST,products,LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[products]
77,istoric/istoricul căutărilor.json,istoricul căutărilor.json,LIST,activityControls,LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[activityControls]
86,istoric/istoricul-vizionărilor.json,istoricul-vizionărilor.json,LIST,header,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[header]
95,istoric/istoricul-vizionărilor.json,istoricul-vizionărilor.json,LIST,title,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[title]
104,istoric/istoricul-vizionărilor.json,istoricul-vizionărilor.json,LIST,titleUrl,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[titleUrl]
116,istoric/istoricul-vizionărilor.json,istoricul-vizionărilor.json,LIST,time,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[time]


FINISH: list_path()
FINISH: clean_and_store()
FILE NAME: YT_structure_youtube_takeout-20250314T090052Z-001.json
<class 'dict'>
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()


/tmp/ipykernel_100495/546882260.py:107: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


,file_path,json_name,col_path_0_LIST,col_path_1,col_path_1_LIST,col_path_2,col_path_2_LIST,col_path_3,col_path_3_LIST,col_path_4,col_path_4_LIST,col_path_5,col_path_5_LIST,col_path_6,col_path_6_LIST,col_path_7,col_path_7_LIST,last_valid_index,data_type,path
30,youtube_takeout/history/watch-history.json,watch-history.json,LIST,header,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[header]
38,youtube_takeout/history/watch-history.json,watch-history.json,LIST,title,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[title]
46,youtube_takeout/history/watch-history.json,watch-history.json,LIST,time,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[time]
54,youtube_takeout/history/watch-history.json,watch-history.json,LIST,products,LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[products]
64,youtube_takeout/history/watch-history.json,watch-history.json,LIST,activityControls,LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[activityControls]
72,youtube_takeout/history/search-history.json,search-history.json,LIST,header,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[header]
81,youtube_takeout/history/search-history.json,search-history.json,LIST,title,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[title]
90,youtube_takeout/history/search-history.json,search-history.json,LIST,titleUrl,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[titleUrl]
99,youtube_takeout/history/search-history.json,search-history.json,LIST,time,NO LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[time]
108,youtube_takeout/history/search-history.json,search-history.json,LIST,products,LIST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,col_path_1_values,string,[products]


FINISH: list_path()
FINISH: clean_and_store()


## Merge all data structures into one schema_df

In [92]:
# Path to the folder containing CSV files
output_path = f"{main_path}Youtube/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


# Concatenate all DataFrames
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# Drop rows that are completely identical across all columns
merged_df = merged_df.drop_duplicates()



### Id creation

In [93]:

merged_df['name'] = merged_df['json_name'].str.replace(".json", "")

id_df = merged_df[['name', 'path', 'file_path']]
id_df = id_df.drop_duplicates()
id_df['id'] = ''

id_df['folder1'] = id_df['file_path'].str.strip('/').str.split('/').str[-2]
id_df['folder2'] = id_df['file_path'].str.strip('/').str.split('/').str[-3]


for n in range(1,4):
    duplicates = id_df[id_df.duplicated(subset='id', keep=False)]

    for i, row in duplicates.iterrows():
        path = row['path']
        name = row['name']  
        folder1 = row['folder1']

        if pd.notna(path):
            
            path =  ast.literal_eval(path)
            
            path_rest = path[-n:]
        

            if not isinstance(path_rest, list):
                path_rest = [path_rest]


            id_list = [folder1] + [name] + path_rest

            new_id = ':'.join(id_list)

        else:

            new_id = name

        id_df.at[i, 'id'] = new_id  



duplicates = id_df[id_df.duplicated(subset='id', keep=False)]

for i, row in duplicates.iterrows():
    folder2 = row['folder2']
    id = row['id']
    new_id =  f'{folder2}:{id}'
    id_df.at[i, 'id'] = new_id  



merged_df = pd.merge(merged_df, id_df, on = ['name', 'path','file_path'], how = 'left')

col = merged_df.pop('id') 
merged_df.insert(0, 'id', col) 


#### ID duplicate flags

In [94]:
merged_df = merged_df.astype(str)
merged_df = merged_df.drop_duplicates()
dup = merged_df[merged_df.duplicated('id', keep= False)]
dup['duplicate_flag'] = 'Yes'
dup = dup[['id', 'duplicate_flag']]
merged_df = pd.merge(merged_df, dup, on = 'id', how = 'left')
merged_df['duplicate_flag'] = merged_df['duplicate_flag'].fillna(value = 'No')

### Clean and Save

In [95]:


keep_columns = ['json_name','id', 'column_name', 
                 'path', 'list_path', 'subfield_path', 
                 'var_type', 'data_type', 'file_path', 'duplicate_flag']


merged_df = merged_df[keep_columns ]

merged_df = merged_df.astype(str)
merged_df = merged_df.drop_duplicates()

# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}Youtube/Final/YT_Merged_structures.csv", index=False)